In [2]:
import numpy as np
from astropy.io import ascii
from pylab import *
from astropy.table import Table, vstack
import pandas as pd
from expecto import get_spectrum
from scipy.interpolate import RegularGridInterpolator as rgi
from itertools import product
from scipy.stats import binned_statistic
#%run Interpolate.ipynb
#%run GetSpectra.ipynb
#%run Binning.ipynb

In [1]:
'''
calculates photon noise, normalized to 0. throughput and transmission are fractional values. Transmission is set up to be
calculated over array of water content values (for pwv timeseries). 

Questions: how do we get initial photon count from the star from the flux? In practice I think we could get that value
directly from Henrietta's CCD detectors, but how do we do it simulation?

I guess if it's normalized, it's really calculting a percentage value and not actual changes in flux, and any flux values
could be used.

What units for exposure time? (pwv time series, each point is 1 minute) (needs to compare to flux, meausured in photons/sec)

re-write in terms of gaussian

after re-writing: I'm getting significantly different values from the gaussian and poisson distributions. It seems like 
poisson dist has much (orders of mag) larger std dev. Maybe I'm using the np version of the random generator wrong?
I thought poisson std dev was supposed to decrease with point count, but it still seems large even with values of 10**14

missing bin size, should be low resolution bin size

Note: right now, bin sizes are in angstroms, since this is the unit that the hi-res flux comes with. Bin sizes are filtered
from binning function to stacking to division_noise to photon_noise. bin sizes for 1.8-1.95 microns are about 180-190 A. 
'''

def photon_noise(flux, bin_size, mirror_diameter = 100, exp_time = 60, throughput = .4):
    
    mirror_area = np.pi*((mirror_diameter/2)**2) - np.pi*((25)**2)
    #print("mirror area:"+str(mirror_area))
    
    #bin_size_array = [bin_array[1]-bin_array[0]]
    '''
    i=0
    for n in bin_array:
        if i == len(bin_array)-1:
            break
        bin_size2 = bin_array[i+1] - bin_array[i]
        bin_size_array.append(bin_size2)
        j+=1
    '''
    num_photons = flux*mirror_area*exp_time*throughput*bin_size
    #print("num photons:"+str(num_photons[0:10]))
    noise_std_dev = 1/(num_photons**(1/2))
    #print("noise std dev: "+str(noise_std_dev))
    
    photon_noise_gaussian = np.random.normal(1, noise_std_dev, len(flux))
    
    return photon_noise_gaussian


'''
throughput of 0.4, swope is 1m in diameter, hole is .5m in diameter
'''

'\nthroughput of 0.4, swope is 1m in diameter, hole is .5m in diameter\n'

In [ ]:
def P_pwv(v, c = 1e-7):

    return c * v**-2

In [ ]:
def angular_correlation(separation):

    return np.exp(-0.5*(separation/45)**2)

In [ ]:
def power_spectrum(N, omega_max, M, timestamps, mean = 0, target = True, phase = None, separation = None):

    delta_omega = omega_max / N
    A_n = np.array([2 * np.sqrt(P_pwv(n * delta_omega + .00001) * delta_omega) for n in range(N + 1)])
    A_n_zero = np.array([0 for n in range(N + 1, M)])
    A_n = np.concatenate((A_n, A_n_zero))

    
    if target:
        phi = np.random.uniform(0,2 * np.pi, M)
        B_n = np.exp(1.0j*phi)
        phase = B_n
        pwv = np.real(np.fft.fft(A_n*B_n)/2)
        
        pwv += mean
        pwv_delta_t = np.pi/omega_max
        pwv_t = np.arange(0, M*pwv_delta_t, pwv_delta_t)/3600
        binned_pwv, binned_pwv_t, _ = binned_statistic(pwv_t, pwv, bins = timestamps)
        binned_pwv = np.insert(binned_pwv, 0, np.mean(binned_pwv))

    else:
        B_n = phase
        phi = np.random.uniform(0, 2*np.pi, M)
        C_n = A_n*angular_correlation(separation)
        D_n = A_n*np.sqrt(1 - angular_correlation(separation)**2)
        E_n = np.exp(1.0j*phi)
        
        pwv = np.real(np.fft.fft(C_n*B_n + D_n*E_n)/2)
        
        pwv += mean
        
        pwv_delta_t = np.pi/omega_max
        pwv_t = np.arange(0, M*pwv_delta_t, pwv_delta_t)/3600
        binned_pwv, binned_pwv_t, _ = binned_statistic(pwv_t, pwv, bins = timestamps)
        binned_pwv = np.insert(binned_pwv, 0, np.mean(binned_pwv))


    return binned_pwv_t, binned_pwv, phase, mean

In [ ]:
'''
if __name__ == "__main__":

    timestamps = np.linspace(0,10,1000) #time in hours
    t, target_star_pwv, target_star_phase, target_star_mean = power_spectrum(12000, 1, 24000, timestamps, mean = 2)
    t1, comparison_star_pwv, _, _ = power_spectrum(12000, 1, 24000, timestamps, mean = target_star_mean, target = False, phase = target_star_phase, separation = 25)

    plt.plot(t, target_star_pwv)
    plt.plot(t, comparison_star_pwv)
    plt.show()
'''